In [1]:
import keras,matplotlib, random, math
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
def binary_to_num(population):
    # dropout rate1------------------------------------
    dp_st = [str(i) for i in population[2][:7]]
    dp_string = str("".join(dp_st))
    dp_num = int(dp_string, 2)
    if dp_num > 80: dp_num = 80
    dropout_rate = float(dp_num /100)
    # dropout rate2------------------------------------
    dp2_st = [str(i) for i in population[2][7:14]]
    dp2_string = str("".join(dp2_st))
    dp2_num = int(dp2_string, 2)
    if dp2_num > 80: dp2_num = 80
    dropout2_rate = float(dp2_num /100)
    # batch size---------------------------------------
    batch_st = [str(i) for i in population[2][14:24]]
    batch_string = str("".join(batch_st))
    batch_num = int(batch_string, 2)
    if batch_num > 1000: batch_num = 1000
    # learning rate-------------------------------------
    lr_st = [str(i) for i in population[2][24:]]
    lr_string = str("".join(lr_st))
    lr_num = int(lr_string, 2)
    if lr_num > 100: lr_num = 100
    elif lr_num == 0: lr_num = 1
    lr_rate = float(lr_num / 1000)
    return dropout_rate, dropout2_rate, batch_num, lr_rate

def fitness(population):
    high_accuracy = 0
    # print(population[1])
    dp1_num, dp2_num, batch_num, lr_rate = binary_to_num(population) 
    accuracy = cnn(dp1_num, dp2_num, batch_num, lr_rate) # 모델 돌려서 accuracy 값 저장
    for i in range(len(accuracy)):
      current_accuracy = round(accuracy[i],4)
      population[1].append(current_accuracy)
      if current_accuracy > high_accuracy: # 가장 높은 accuracy 값 저장
        high_accuracy = current_accuracy
    population= (high_accuracy, population[1], population[2])
    return population


def cnn(dp1_rate, dp2_rate, bat_size, lr_rate):
  num_classes = 10 #맞출수 있는 레이블 개수
  epochs = 10

  # fashion mnist train 60000, test 1000개 있다
  (xf_train, yf_train), (xf_test, yf_test) = fashion_mnist.load_data()
  # resize the image size
  xf_train = xf_train.reshape((xf_train.shape[0], 28, 28, 1))
  xf_test = xf_test.reshape((xf_test.shape[0], 28, 28, 1))
  input_shape = (28, 28, 1)
  # 픽셀의 intensities는 [0,1] 범위로 조정
  xf_train = xf_train.astype("float32") / 255.0
  xf_test = xf_test.astype("float32") / 255.0
  # one-hot encode the training and testing lables
  yf_train = keras.utils.to_categorical(yf_train, num_classes)
  yf_test = keras.utils.to_categorical(yf_test, num_classes)

  #모델만들기 
  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3,3),
                  activation='relu',
                  input_shape = input_shape)) #(28, 28)
  model.add(Conv2D(64, (3,3), activation = 'relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(dp1_rate)) #여기서 사용-----------------------------------
  model.add(Flatten())
  model.add(Dense(128, activation = 'relu'))
  model.add(Dropout(dp2_rate)) #여기서 사용------------------------------------
  model.add(Dense(num_classes, activation = 'softmax')) #10개의 레이블
  # model.summary()

  model.compile(loss = "categorical_crossentropy",
              optimizer = keras.optimizers.Adadelta(learning_rate=lr_rate), #여기서 사용---------------
              metrics = ['accuracy'])
  
  history = model.fit(xf_train,yf_train,
                      batch_size = bat_size, #여기서 사용-----------------------
                      epochs = epochs, 
                      verbose = 1,
                      validation_data = (xf_test, yf_test))

  new = list(history.history['val_accuracy'])
  return(new)

def selection(population, k):
  tournament_pool = random.sample(population, k)
  result = sorted(tournament_pool, key=lambda x:x[0], reverse=True)
  return result[0]

def one_point_crossover(p1, p2):
  if np.array_equal(p1[2], p2[2]):
    return (0, [], p1[2]), (0, [], p2[2])
  else:
    length = len(p1[2])
    cut = random.randint(0, length-1)
    o1 = p1[2][:cut]
    o1.extend(p2[2][cut:])
    o2 = p2[2][:cut]
    o2.extend(p1[2][cut:])
  return (0, [], o1), (0, [], o2)

def two_point_crossover(p1, p2):
  if np.array_equal(p1[2], p2[2]):
    return (0, [], p1[2]), (0, [], p2[2])
  else:
    length = len(p1[2])
    cut1 = random.randint(0, length-1)
    cut2 = random.randint(0, length-1)
    if cut1 < cut2:
      cut1 = cut1
      cut2 = cut2
    elif cut1 > cut2:
      save = cut1
      cut1 = cut2
      cut2 = save
    else:
      pass
    first1, second1, third1 = p1[2][:cut1], p2[2][cut1:cut2], p1[2][cut2:]
    first2, second2, third2 = p2[2][:cut1], p1[2][cut1:cut2], p2[2][cut2:]
    o1 = first1 + second1 + third1
    o2 = first2 + second2 + third2
  return (0, [], o1), (0, [], o2)

def uniform_crossover(p1, p2):
  if np.array_equal(p1[2], p2[2]):
    return (0, [], p1[2]), (0, [], p2[2])
  else:
    length = len(p1[2])
    o1=[]
    o2=[]
    for i in range(length):
      if random.random() < 0.5:
        o1.append(p1[2][i])
        o2.append(p2[2][i])
      else:
        o1.append(p2[2][i])
        o2.append(p1[2][i])
  return (0, [], o1), (0, [], o2)


def bit_inversion_mutation(offspring, rate):
  for i in range(len(offspring[2])):
    if random.random() < rate:
      if offspring[2][i] ==1:
        offspring[2][i]=0
      else:
        offspring[2][i]=1
  return offspring

def scramble_mutation(offspring, rate):
  if random.random() < rate:
    length = len(offspring[2])
    a = random.randint(0, length - 1)
    b = random.randint(0, length - 1)
    if a < b:
      a = a
      b = b
    elif a > b:
      save = a
      a = b
      b = save
    else:
        pass
    first, second, third = offspring[2][:a], offspring[2][a:b], offspring[2][b:]
    new_list = first + random.sample(second, len(second)) + third
    offspring = (offspring[0], offspring[1], new_list)
  return offspring

def inversion_mutation(offspring, rate):
  if random.random() < rate:
    length = len(offspring[2])
    a = random.randint(0, length - 1)
    b = random.randint(0, length - 1)
    if a < b:
      a = a
      b = b
    elif a > b:
      save = a
      a = b
      b = save
    else:
        pass
    first, second, third = offspring[2][:a], offspring[2][a:b], offspring[2][b:]
    new_list = first + second[::-1] + third
    offspring = (offspring[0], offspring[1], new_list)
  return offspring


def ga(pop, gen):
    population = []
    for i in range(pop):
        randomBinaryNumbers = np.random.randint(2, size=31) # 24bit
        accuracy_list =[] # validation accuracy 저장할 리스트
        binary_list = (0, accuracy_list, list(randomBinaryNumbers)) #list로 변환
        fit = fitness(binary_list) # 각 chromosome조합으로 얻은 accuracy
        population.append(fit)

    for g in range(gen):
      offspring =[]
      while len(offspring) < pop:
        parent1 = selection(population, 3)
        parent2 = selection(population, 3)

        # offspring1, offspring2 = one_point_crossover(parent1, parent2) # 잘못걸리면..5시간
        offspring1, offspring2 = two_point_crossover(parent1, parent2) #시간이 좀더 빠름 길면 3시간 평균 2시간
        # offspring1, offspring2 = uniform_crossover(parent1, parent2) #결과는 제일 좋으나..3시간에서 6시간도 걸린다..평균 4-5시간 

        #3개의 crossover의 기준으로 사용---------------------------------------
        # offspring1 = bit_inversion_mutation(offspring1, 0.1) #result1_1 (파일 저장 이름)
        # offspring2 = bit_inversion_mutation(offspring2, 0.1)

        # offspring1 = bit_inversion_mutation(offspring1, 0.3) #result3_1
        # offspring2 = bit_inversion_mutation(offspring2, 0.3)

        # offspring1 = bit_inversion_mutation(offspring1, 0.6) #result6_1
        # offspring2 = bit_inversion_mutation(offspring2, 0.6)

        # two-points crossover를 중심으로 사용----------------------------------
        # offspring1 = scramble_mutation(offspring1, 0.1) #scramble1_result1_1
        # offspring2 = scramble_mutation(offspring2, 0.1)

        # offspring1 = scramble_mutation(offspring1, 0.3) #scramble3_result1_1
        # offspring2 = scramble_mutation(offspring2, 0.3)

        # offspring1 = scramble_mutation(offspring1, 0.6) #scramble6_result1_1
        # offspring2 = scramble_mutation(offspring2, 0.6)

        # offspring1 = inversion_mutation(offspring1, 0.1) #inversion1_result1_1
        # offspring2 = inversion_mutation(offspring2, 0.1)

        # offspring1 = inversion_mutation(offspring1, 0.3) #inversion3_result1_1
        # offspring2 = inversion_mutation(offspring2, 0.3)

        offspring1 = inversion_mutation(offspring1, 0.6) #inversion6_result1_1
        offspring2 = inversion_mutation(offspring2, 0.6)

        offspring1 = fitness(offspring1)
        offspring2 = fitness(offspring2)

        offspring.append(offspring1)
        offspring.append(offspring2)
      pool = population
      pool.extend(offspring)
      pool = sorted(pool, key = lambda x: x[0], reverse=True)
      population = pool[:pop]
    print("final population ", population)

    final=[]
    for j in range(pop):
      dp_st = [str(i) for i in population[j][2][:7]]
      dp_string = str("".join(dp_st))
      dp_num = int(dp_string, 2)
      if dp_num > 80: dp_num = 80
      dropout_rate = float(dp_num /100)
      # dropout rate2------------------------------------
      dp2_st = [str(i) for i in population[j][2][7:14]]
      dp2_string = str("".join(dp2_st))
      dp2_num = int(dp2_string, 2)
      if dp2_num > 80: dp2_num = 80
      dropout2_rate = float(dp2_num /100)
      # batch size---------------------------------------
      batch_st = [str(i) for i in population[j][2][14:24]]
      batch_string = str("".join(batch_st))
      batch_num = int(batch_string, 2)
      if batch_num > 1000: batch_num = 1000
      # learning rate-------------------------------------
      lr_st = [str(i) for i in population[j][2][24:]]
      lr_string = str("".join(lr_st))
      lr_num = int(lr_string, 2)
      if lr_num > 100: lr_num = 100
      elif lr_num == 0: lr_num = 1
      lr_rate = float(lr_num / 1000)
      savee= (population[j][0], population[j][1], (dropout_rate, dropout2_rate, batch_num, lr_rate))
      final.append(savee)

    drive.mount('/content/drive')
    df = pd.DataFrame(final)
    #자신의 Google Drive와 연결: SEP592_project파일을 My Drive에 만들어 주어야한다
    df.to_csv('/content/drive/My Drive/SEP592_project/inversion6_result1_1.csv', header=None, index=None)

if __name__ == '__main__':
    ga(10, 15)

4423680/4422102 [==============================] - 1s 0us/step
Epoch 1/10
101/101 [==============================] - 2s 17ms/step - loss: 2.2686 - accuracy: 0.1509 - val_loss: 2.1676 - val_accuracy: 0.5060
Epoch 2/10
101/101 [==============================] - 2s 15ms/step - loss: 2.1265 - accuracy: 0.2548 - val_loss: 1.9047 - val_accuracy: 0.6281
Epoch 3/10
101/101 [==============================] - 2s 15ms/step - loss: 1.9054 - accuracy: 0.3390 - val_loss: 1.5584 - val_accuracy: 0.6053
Epoch 4/10
101/101 [==============================] - 2s 15ms/step - loss: 1.6986 - accuracy: 0.3975 - val_loss: 1.3181 - val_accuracy: 0.6230
Epoch 5/10
101/101 [==============================] - 2s 15ms/step - loss: 1.5539 - accuracy: 0.4428 - val_loss: 1.1768 - val_accuracy: 0.6311
Epoch 6/10
101/101 [==============================] - 2s 15ms/step - loss: 1.4411 - accuracy: 0.4794 - val_loss: 1.0772 - val_accuracy: 0.6523
Epoch 7/10
101/101 [==============================] - 2s 15ms/step - loss: 1.36